In [1]:
import numpy as np

def reweight_distribution(original_distribution, temprature=0.5):
    distribution = np.log(original_distribution) / temprature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [2]:
import keras

text = open('nietzsche.txt').read().lower()

print('Corpus length : ', len(text))

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Corpus length :  600901


In [3]:
maxlen = 60 
step = 3
sentances = []
next_char = []

for i in range(0, len(text)-maxlen, step):
    sentances.append(text[i: i+maxlen])
    next_char.append(text[i+maxlen])
    
print ('Number of sequences : ', len(sentances))

chars = sorted(list(set(text)))
print('Unique characters : ', len(chars))
char_index = dict((char, chars.index(char)) for char in chars)

print('Vectrization...')

x = np.zeros((len(sentances), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentances), len(chars)), dtype=np.bool)

for i, sentance in enumerate(sentances):
    for t, char in enumerate(sentance):
        x[i, t, char_index[char]] = 1
    y[i, char_index[next_char[i]]] = 1
    
print('Vectrization completed .')

Number of sequences :  200281
Unique characters :  59
Vectrization...
Vectrization completed .


In [5]:
from keras import layers

model =  keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr= 0.01) #bad learning rate caused loss to explode
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

In [5]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype(np.float64)
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
import random
import sys

for epoch in range(60):
    print('\n\nepoch : ',epoch+1)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = np.random.randint(0, len(text)-maxlen-1)
    generated_text = text[start_index: start_index+maxlen]
    print('---Generating with seed : "'+generated_text+'"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('\n---Temperature : ', temperature)
        sys.stdout.write(generated_text)
        
        aux_text = generated_text
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(aux_text):
                sampled[0, t, char_index[char]] = 1
            
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            aux_text += next_char
            aux_text = aux_text[1:]
            
            sys.stdout.write(next_char)


epoch :  1
Epoch 1/1
200281/200281 [==============================] - 284s 1ms/step - loss: 1.9784
---Generating with seed : "ost subtle
sense of the term: the strong man will necessaril"

---Temperature :  0.2
ost subtle
sense of the term: the strong man will necessarily and the precised to the serition of the precised to the present the conscious and which as the condicions of the precised to the sermanity of the preciation of the perpare the perples the condicions of the precise of the precise of the preasure it is the concertion to the precise the consertion of the prives and the precise the concertion of the perples the consent of the precise of the precise 
---Temperature :  0.5
ost subtle
sense of the term: the strong man will necessarily into which are constill and conscience of the self-sumply of hit dissine of the mand of experient and many discience of the some
like oprention of the conscious the contery of the spirit of the some to charad and feel and precisely inthe condi

give,
necessaal,
peoplesup, by a unwhos
a, thus ilsy of softeul ourselves and tactuent, the valictol "disinyts
experiess
(aculything as other tois that happe to eass, resust 
 niment, believe rawes" rysulh that the emile, theief withis. flonoser: he pmoss to matter to
its back their,
co
epoch :  5
Epoch 1/1
200281/200281 [==============================] - 291s 1ms/step - loss: 1.4653
---Generating with seed : "do not admit of
being despised. according to slave-morality,"

---Temperature :  0.2
do not admit of
being despised. according to slave-morality, and the consideration and the the man is the propers and the self-period the consequently and the delicate been the things and the conscious and the profound the consideration of the conscious and such a something the consequently in the consideration of the self-consideration of the called before the world and the struggle, and the consideration of the conscious and the being to the self-conditi
---Temperature :  0.5
do not admit of
be

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


ght evilven ununrsil in . 
epoch :  12
Epoch 1/1
200281/200281 [==============================] - 289s 1ms/step - loss: 1.3757
---Generating with seed : "osition is as clear as sunlight, and
yet here everyone prefe"

---Temperature :  0.2
osition is as clear as sunlight, and
yet here everyone preferred the self-extent and distinction of the experiences of the sense of the strengthed the strengt the sense of the sense of the sense and contradiction of the self-continuant that in the sense of the fact the experiences of the proper in the sense of the strengthes of the concerning the sense and distinction of the religious and contrary of the self-experience and such a strong of the sense and s
---Temperature :  0.5
osition is as clear as sunlight, and
yet here everyone prefert the experiences of the privoric of the moral physiologing impulse of the intellected the sundery the sense and concerning the will of philosophers of which the moral, seem that whom the what in it. why are not prove

not would are out more in the valuation. the will betrays poor, he can things, and "for our knowledge course, and misuponding of science even there most find the grad some uncluptions--as an amflike--only sufforer that hallequally debidnest servitoned innoal
---Temperature :  1.2
 still be original, probably as parodists of
the world's hister of nequentlats), the ambe: soris everything strang!-ony
ar. what doubt, more, of grut: one does s artuad
"immented-t
cleuse such todagity? on )justih becaulded in which "centurie unfen of ithistual advendealing
to self--and
. burning spill, scaility."
as deaing necesnating commander:
as ts are speaks grans spiriously can being eld ember, outuroul: ther, of fecurious
spense
or tome
do with is a
epoch :  20
Epoch 1/1
200281/200281 [==============================] - 293s 1ms/step - loss: 1.3367
---Generating with seed : "1

it is often enough, and always with great surprise, intim"

---Temperature :  0.2
1

it is often enough, and always with great s

retarded and more soul, that the subject the sensual the soul in the sentiments that it is not be a sort of the soul of the contempted to be a development and the sense and the standard of the contemplation of the same the standard of the schopenhaue and the standard of the personal to the same time the same the sense and man than the standing the spirit of the conditions of the same the sense to the standard of
---Temperature :  0.5
-the development of mankind has been longest
retarded and most nature. the type of the motives of the ascent to the opposite the most class to the spirit in the fundarnies of the sense in him, in all the transfrom and divine the end in moral men of the point of the moral have always that it is experienced and philosophy of the pathoue a constantly in the such
distinction of subtle by the constitutes the minded that a that it is yet strength, the sense, the
---Temperature :  1.0
-the development of mankind has been longest
retarded and more negests: re"itga

are occasioned, and presentidy
power-ones a ricay: too-cystomical decaint
enviroor before its in""aquedly regaid-know, the distines but the oloncen,
as -alat out pedifore is ; he has old
classelar bound and excess
in its its
that fortunached in all the will
of waity, if only ono
ruling rich, topegrive civecus of which even conmains a  avjums.)
wither man of
confers ansic gitliprang eve. be. whither is conc.at ertony,
epoch :  35
Epoch 1/1
200281/200281 [==============================] - 295s 1ms/step - loss: 1.3098
---Generating with seed : "inexorably "i am morality itself and nothing else
is moralit"

---Temperature :  0.2
inexorably "i am morality itself and nothing else
is moralities of the same time and indifferent and also the same time the same time a man is also all the same strength of the same time and the strength, the same taste of the same time and the same time and the present and indignants of the strength of the same time the most soul, the same time is also the sense o

therefrong have a great abortoners
inextravage.
they will withly
exampl and account of result noisful, to tested place they were has weiad, everythabling sweep philosopher by richers,
certaisss and dwafgical intenlourac-ibilitume
---Temperature :  1.2
house-animal, and knows only the wants of a house-animal (like's years whiche sefircts of chrimal, obediently us in which is there, no philosoph in.ry as a something highest desire of nothing the facts, permited nevery must, the facts, explanative of pour of though the roseed the way. class aspeciality of humbmansy
against its man russion? i impureass," now deformed at
that wa on the daveoust oseckestly: now life being overlax, oe may
! without the
eeds-el
epoch :  39
Epoch 1/1
200281/200281 [==============================] - 296s 1ms/step - loss: 1.3052
---Generating with seed : " this point, is not
the case) this fact would constitute an "

---Temperature :  0.2
 this point, is not
the case) this fact would constitute an extent that the

one does--this is the charmt to be man is the same and the same the same the same the same to the sense of the sense of the same in the same the same to be still and the same the same the same the same the same the same in the same the same to be as it is the same the same to the same as the same the same to have the same the same the same the same the same the soul, the same and the seriously in the same and the 
---Temperature :  0.5
re of that personality in mind in all that
one does--this is a possible to the gregarity, in so as a petty and false
the same the sound to it
which the sacress of the same condity, they are still have to this to reason and such artful believe in the philosophy in the same and spiritual possess and all this one then there was such absurd to be still in
the pasially something in the strength has not been a commonscience, the soul the can say that it is the b
---Temperature :  1.0
re of that personality in mind in all that
one does--this is to their "must n

the lap of beaatert eiteaar ithit itrat o sathera itaoire ane koruee nterethi iniarie tit tae thtetiititoe athori tie re araittitithit iale ia t ttieth et t ea ee ar ti ithitetatitithorare o nat t tlat titit t thetot ite teroiottoer oateitieanertieon thiee io tha rt aatuioriteit ane thee qoras at ty erto tariatire i ateeasrit ee it ist oe otoretet anr eteeitiit taojh retor ato  reotiean orithta iotoaoit en our
---Temperature :  1.2
, they cannot have their source. but rather in
the lap of be itiio erarioeerato ortit ei ieiaeoren stinare a'aae.uo jutrritititifiitio ijs he siarthnieruht ha est iterairoororitit aria iatitort eththaastit tthe t tir ant  asitooe eortee  etariasnaore orat tieint qoratar alallthitorr lh sulit te anea toaate riottour ai ois orf t arat grta tite t t tiae ae
ithir  thate ant  ao a atioer t intt ait thtaor iietro iite    hoaee thtaeortatheeeorariith eeot ilo 4t
epoch :  54
Epoch 1/1
200281/200281 [==============================] - 288s 1ms/step - loss: nan
---Gen